In [1]:
from datetime import date
from nsepy import get_history
import pandas as pd

In [2]:
def obtain_data(ticker,start,end):
# Enter the start and end dates using the method date(yyyy,m,dd)    
    stock=get_history(symbol=ticker,start=start,end=end)
    df=stock.copy()
    df=df.reset_index()
    df=df.drop(['Series','Prev Close','Last','Turnover','%Deliverble','Trades'],axis=1)
    df=df.rename({'Open':'open_price','Close':'close_price','High':'high','Low':'low','Volume':'volume'},axis='columns')
    df.index=df.Date
    return df

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
"""This cell defineds the plot_candles function"""

def plot_candles(pricing, title=None, volume_bars=False, color_function=None, technicals=None):
    """ Plots a candlestick chart using quantopian pricing data.
    
    Author: Daniel Treiman
    
    Args:
      pricing: A pandas dataframe with columns ['open_price', 'close_price', 'high', 'low', 'volume']
      title: An optional title for the chart
      volume_bars: If True, plots volume bars
      color_function: A function which, given a row index and price series, returns a candle color.
      technicals: A list of additional data series to add to the chart.  Must be the same length as pricing.
    """
    def default_color(index, open_price, close_price, low, high):
        return 'r' if open_price[index] > close_price[index] else 'g'
    color_function = color_function or default_color
    technicals = technicals or []
    open_price = pricing['open_price']
    close_price = pricing['close_price']
    low = pricing['low']
    high = pricing['high']
    oc_min = pd.concat([open_price, close_price], axis=1).min(axis=1)
    oc_max = pd.concat([open_price, close_price], axis=1).max(axis=1)
    
    if volume_bars:
        fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [3,1]},figsize=(7,7))
    else:
        fig, ax1 = plt.subplots(1, 1)
    if title:
        ax1.set_title(title)
    fig.tight_layout()
    x = np.arange(len(pricing))
    candle_colors = [color_function(i, open_price, close_price, low, high) for i in x]
    candles = ax1.bar(x, oc_max-oc_min, bottom=oc_min, color=candle_colors, linewidth=0)
    lines = ax1.vlines(x , low, high, color=candle_colors, linewidth=1)
    ax1.xaxis.grid(True)
    ax1.yaxis.grid(True)
    ax1.xaxis.set_tick_params(which='major', length=3.0, direction='in', top='off')
    ax1.set_yticklabels([])
    # Assume minute frequency if first two bars are in the same day.
    frequency = 'minute' if (pricing.index[1] - pricing.index[0]).days == 0 else 'day'
    time_format = '%d-%m-%Y'
    if frequency == 'minute':
        time_format = '%H:%M'
    # Set X axis tick labels.
    #plt.xticks(x, [date.strftime(time_format) for date in pricing.index], rotation='vertical')
    for indicator in technicals:
        ax1.plot(x, indicator)
    
    if volume_bars:
        volume = pricing['volume']
        volume_scale = None
        scaled_volume = volume
        if volume.max() > 1000000:
            volume_scale = 'M'
            scaled_volume = volume / 1000000
        elif volume.max() > 1000:
            volume_scale = 'K'
            scaled_volume = volume / 1000
        ax2.bar(x, scaled_volume, color=candle_colors)
        volume_title = 'Volume'
        if volume_scale:
            volume_title = 'Volume (%s)' % volume_scale
        #ax2.set_title(volume_title)
        ax2.xaxis.grid(True)
        ax2.set_yticklabels([])
        ax2.set_xticklabels([])
    return fig    

In [7]:
fig=plot_candles(df_pricing,volume_bars=True)

NameError: ignored

In [8]:
fig.savefig('workspace/share_market/data/Testing.jpeg',dpi=70)

NameError: ignored

In [25]:
df=df_pricing.copy()
df=df.reset_index(drop=True)

In [ ]:
equities=['BAJFINANCE']
# /'RELIANCE','INFY','HDFC','HDFCBANK','HDFCLIFE','ZEEL','DELTACORP','ITC','ASIANPAINT'
for equity in equities :
    df_pricing=obtain_data(equity,date(2018,1,1),date(2020,9,1))
    df=df_pricing.copy()
    df=df.reset_index(drop=True)
    n_days=5
    fraction_movement=0.037
    df['Trend']=None
    for i in range(len(df)):
        try :
            for n in range(n_days): 
                if  df.loc[i,'close_price'] - df.loc[i+n,'close_price'] >= fraction_movement*df.loc[i,'close_price']:
                    df.loc[i,'Trend'] ='Down'
                    if i >=20 :
                        fig=plot_candles(df_pricing[i-20:i],volume_bars=True)
                        fig.savefig('workspace/share_market/data/Down/{0}{1}.png'.format(df_pricing['Symbol'][i],i),dpi=70)
                    print('Down',i,n)
                    break
                elif df.loc[i+n,'close_price'] - df.loc[i,'close_price'] >= fraction_movement*df.loc[i,'close_price'] :
                    df.loc[i,'Trend']='Up'
                    if i >20 :    
                        fig=plot_candles(df_pricing[i-20:i],volume_bars=True)
                        fig.savefig('workspace/share_market/data/Up/{0}{1}.png'.format(df_pricing['Symbol'][i],i),dpi=70)
                    print('Up',i,n)
                    break
                else :
                    df.loc[i,'Trend']= 'No Trend'
        except :
            print(i)
            pass

In [11]:
df_pricing['Symbol'][i]

'BAJFINANCE'

In [12]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [13]:
from fastai.vision import *
from fastai.metrics import error_rate

In [14]:
bs = 64
# bs = 16 

In [15]:
from pathlib import Path


In [17]:
path=Path('workspace/share_market/data')
path_save=Path('workspace/share_market/data/Processed')

In [102]:
path.ls()


(#3) [Path('workspace/share_market/data/Up'),Path('workspace/share_market/data/.ipynb_checkpoints'),Path('workspace/share_market/data/Down')]

In [59]:
from fastai import *
from fastai.vision.augment import aug_transforms, RandomResizedCrop, Resize, FlipItem
from fastai.data.transforms import get_image_files, Normalize, RandomSplitter, GrandparentSplitter, RegexLabeller, ToTensor, IntToFloatTensor, Categorize, parent_label
from fastai.vision import *
from fastai.callback.schedule import lr_find, fit_one_cycle
from fastai.learner import Learner

from fastai.metrics import error_rate

In [56]:
from fastai.vision.data import ImageDataLoaders
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.vision import *



In [83]:
from torch import nn
from torchvision.models.resnet import resnet34
from fastai.vision.augment import aug_transforms, RandomResizedCrop, Resize, FlipItem
from fastai.vision.core import PILImage, imagenet_stats
from fastai.vision.data import ImageBlock
from fastai.vision.all import cnn_learner
from fastai.vision.learner import vision_learner, create_head, create_body, num_features_model, default_split, has_pool_type, apply_init, _update_first_layer

In [103]:
np.random.seed(42)
data = ImageDataLoaders.from_folder(path, train=".", valid_pct=0.2, size=224, num_workers=4,
                                     ds_tfms=aug_transforms(flip_vert=False, max_lighting=0.1, max_zoom=1.05, max_warp=0.,max_rotate=3)
                                    )
# np.random.seed(42)
# data = ImageDataLoaders.from_folder(path, train=".", valid_pct=0.2,
#         ds_tfms=get_transforms(flip_vert=False, max_lighting=0.1, max_zoom=1.05, max_warp=0.,max_rotate=3), size=224, num_workers=4).normali

In [107]:
learn = vision_learner(data, resnet34, metrics=error_rate)


In [108]:
learn.fit_one_cycle(4)


epoch,train_loss,valid_loss,error_rate,time
0,1.056637,0.901741,0.448276,06:36
1,1.008060,0.791098,0.396552,06:27
2,0.866649,0.773702,0.413793,06:18
3,0.782700,0.782754,0.465517,06:20


In [109]:
learn.unfreeze()


In [111]:
learn.lr_find()


Traceback (most recent call last):
  File "/Users/vipinv/opt/anaconda3/lib/python3.9/multiprocessing/queues.py", line 251, in _feed
    send_bytes(obj)
  File "/Users/vipinv/opt/anaconda3/lib/python3.9/multiprocessing/connection.py", line 205, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/Users/vipinv/opt/anaconda3/lib/python3.9/multiprocessing/connection.py", line 416, in _send_bytes
    self._send(header + buf)
  File "/Users/vipinv/opt/anaconda3/lib/python3.9/multiprocessing/connection.py", line 373, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: ignored

In [ ]:
learn.recorder.plot()


In [ ]:
learn.fit_one_cycle(10, max_lr=slice(1e-6,1e-4))


In [96]:
learn.lr_find()


TypeError: ignored

In [ ]:
learn.recorder.plot()


In [ ]:
learn.fit_one_cycle(5, max_lr=slice(1e-6,1e-4))


In [ ]:
learn.lr_find()


In [ ]:
learn.recorder.plot()


In [ ]:
learn.fit_one_cycle(5,max_lr=slice(1e-5,1e-4))


In [ ]:
learn.save('First Model')


In [ ]:
data = ImageDataLoaders.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(flip_vert=False, max_lighting=0.1, max_zoom=1.05, max_warp=0.,max_rotate=3), size=352, num_workers=4)

In [ ]:
learn.data=data


In [ ]:
gc.collect()
learn.lr_find()

In [ ]:
learn.recorder.plot()


In [ ]:
learn.fit_one_cycle(5,max_lr=slice(1e-5,1e-4))


In [ ]:
learn.lr_find()


In [ ]:
learn.recorder.plot()


In [ ]:
learn.fit_one_cycle(5,max_lr=slice(1e-6,1e-5))


In [ ]:
learn.save('Model 2')

In [ ]:
def show_heatmap(hm):
    fig,ax = plt.subplots()
    xb_im.show(ax)
    ax.imshow(hm, alpha=0.6, extent=(0,352,352,0),
              interpolation='bilinear', cmap='magma');
    return fig

In [ ]:
answer=show_heatmap(avg_acts)
answer.savefig(path/'trial.png')